In [ ]:
import pandas as pd
import bamboolib
import os
import datetime
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.ticker import MultipleLocator, FixedFormatter, FixedLocator

import seaborn as sns
import scipy
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from scipy.spatial.distance import squareform
from IPython.core.display import display, HTML
import re

In [ ]:
current_directory = os.getcwd()
print(current_directory)

## Analyse des segments

In [ ]:
aymeric =  "/home/aymeric/python-scripts/anses_medialab/datas/" #aymeric
jp = '~/Dropbox/Mac/Desktop/CRD Anses/all3/' # Jean Philippe
jp_index = '~/Dropbox/Mac/Desktop/CRD Anses/code/indexation_results/' # Jean Philippe index

if 'aymeric' in current_directory:
    path_base = aymeric
    index=f"{path_base}index_allall_domainsexhaustive.csv"
elif 'Mac' in current_directory:
    path_base = jp
    index=f'{jp_index}index_allall_domainsexhaustive.csv'
elif 'd:/Projects' in current_directory:
    path_base = "d:/Projects/Medialab/"
    index=f"{path_base}index_allall_domainsexhaustive.csv"

file_segmentation = f"{path_base}segmentation_common_freq.csv"
data_file = path_base+'fb_tw_med_data.csv.gz'#,line_terminator='\n',index=False)


all_corpus = f"{path_base}all_corpus_seg_with_sdhi_PI.csv"
cluster_qualification = f"{path_base}cluster_qualification.csv"

#twittos = f"{path_base}tweets_pesticides/500_first_twittos_on_pesticides.csv"




In [ ]:
list_base = ["index", "all_corpus"]
which_base = list_base[0]

In [ ]:
dfseg0 = pd.read_csv(file_segmentation, sep = "\t")
df_cluster = pd.read_csv(cluster_qualification, sep = "\t")

if which_base == "all_corpus":
    df0 = pd.read_csv(all_corpus, sep = "\t")
else:
    df_index = pd.read_csv(index, sep = "\t")

    df_index['date'] = pd.to_datetime(df_index['date'], infer_datetime_format=True)
    df_index['yearmonth']=(df_index['date'].dt.strftime('%Y-%m'))
    df_index['date'] = df_index['date'].dt.date
    
    segment = dfseg0[["yearmonth", "segm", "origin"]]
    df0 = df_index.merge(segment, how = "inner", on = ["origin", "yearmonth"])
    df0["start_segment"] = df0.groupby(['origin','segm'])["date"].transform('min')
    df0["end_segment"] = df0.groupby(['origin','segm'])["date"].transform('max')

In [ ]:
dft = pd.read_csv(data_file)


In [ ]:
# Step: Rename column
dft = dft.rename(columns={'id': 'doc_id'})

dft['date'] = pd.to_datetime(dft['date'], infer_datetime_format=True)

dft['yearmonth']=(dft['date'].dt.strftime('%Y-%m'))
dft['date'] = dft['date'].dt.date
    
segment = dfseg0[["yearmonth", "segm", "origin"]]
dft0 = dft.merge(segment, how = "inner", on = ["origin", "yearmonth"])
dft0["start_segment"] = dft0.groupby(['origin','segm'])["date"].transform('min')
dft0["end_segment"] = dft0.groupby(['origin','segm'])["date"].transform('max')



In [ ]:
dfseg0

## Facebook

In [ ]:
def filter_data(data, domain, segment):
    print(domain)
    dfseg = data[0]
    dfseg1 = dfseg.loc[(dfseg["origin"] == domain) & (dfseg["segm"] == segment)]
    
    df = data[1]
    df1 = df.loc[(df["origin"] == domain) & (df["segm"] == segment)]
    
    dft = data[2]
    dft1 = dft.loc[(dft["origin"] == domain) & (dft["segm"] == segment)]
    dft1['doc_id'] = dft1['doc_id'].astype('string')
    
    specific_terms = [x.strip() for x in dfseg1["top50_term_chi2_True"].iloc[0].split(",")]

    print(specific_terms)
    df1.loc[df1["term"].isin(specific_terms) == True, "in_top50_term"] = 1
    df1.loc[df1["term"].isin(specific_terms) == False, "in_top50_term"] = 0

    
    df2 = df1.drop_duplicates(subset = ["doc_id", "term"]).groupby(['doc_id', 'source', "date"]).agg(sum_of_top50_term =('in_top50_term', 'sum')).reset_index()
    df2['doc_id'] = df2['doc_id'].astype('string')
    df2 = df2.merge(dft1[["doc_id", "text"]], how = "inner", on = ["doc_id"])
    
    return dfseg1, df1, dft1, df2, specific_terms

In [ ]:
domains = ["facebook", "twitter", "media"]
segment = 1
domain = domains[0]

list_df = [dfseg0, df0, dft0]
dfseg1, df1, dft1, df2, specific_terms = filter_data(list_df, domain = domain, segment = segment)

In [ ]:
df2

In [ ]:
moyenne = np.mean(df2["sum_of_top50_term"]) 
df3 = df2.loc[df2["sum_of_top50_term"] >= 1 ]
df3 = df3.sort_values(by=['date'], ascending=[True])

#df3 = df3.drop_duplicates(subset = "text")

## Matrice de similarité

In [ ]:
df11 = df1.groupby(['term', 'doc_id']).agg(count_doc = ("doc_id", "count")).reset_index()
df11=df11.merge(df1[['term','in_top50_term']].drop_duplicates(), on='term', how = 'left')
df11=df11.loc[df11["in_top50_term"] ==  1]

In [ ]:
import math


dico_freq={}
for l in df11.groupby('term'):
    term=l[0]
    #print (term)
    dico_freq[term]={}
    for d in l[1].groupby('doc_id'):
        #print (d[0],d[1]['count_doc'].values[0])
        dico_freq[term][d[0]]=d[1]['count_doc'].values[0]

for t in dico_freq:
    N=sum(dico_freq[t].values())
    for d in dico_freq[t]:
        dico_freq[t][d]=dico_freq[t][d]/N
    
                
squared={}
spec_words=list(dico_freq.keys())
for w1 in spec_words:
    s=0
    for d1 in dico_freq[w1]:
        #if d1 in spec_words[w2]:
        s += dico_freq[w1][d1] * dico_freq[w1].get(d1,0)
    squared[w1]=s

similarity=[]
spec_words=list(dico_freq.keys())
for w1 in spec_words:
    sim=[]
    for w2 in spec_words:
        s=0
        for d1 in dico_freq[w1]:
            #if d1 in spec_words[w2]:
            s += dico_freq[w1][d1] * dico_freq[w2].get(d1,0)
        sim.append(s/math.sqrt(squared[w1]*squared[w2]))
    similarity.append(sim)

#similarity

In [ ]:
list_name = [x for x in df11.term.unique()]
print(list_name)

In [ ]:
from matplotlib import pyplot as plt


cmap_reversed = plt.cm.get_cmap('spring_r')
#sns.clustermap(rho_terms)
ax=sns.clustermap(similarity,figsize=(15,15), cmap = cmap_reversed, linewidths=2, linecolor='white', vmax = 0.8)#,labels=range(14))
new_labels=[]
labels_list= list_name
for l in ax.ax_heatmap.axes.get_xticklabels():
    l.set_text(labels_list[int(l.get_text())])
    new_labels.append(l)
ax.ax_heatmap.axes.set_yticklabels(new_labels,rotation=0)
ax.ax_heatmap.axes.set_xticklabels(new_labels,rotation = 90)

#ax.setp(g.ax_heatmap.get_yticklabels(), rotation=0)
#ax.set_xticklabels(ax.get_xticklabels(),rotation = 30)
plt.tight_layout()

plt.savefig(f"clustermap/cm_{domain}_{segment}.png")

## Détermination du nombre de clusters avec kneed Locator

In [ ]:
fig = plt.figure(1, figsize=(15, 5))
row_linkage = linkage(sp.distance.pdist(similarity), method='complete')# for x in (t.values, rho_terms.values.T))
nbclust_max=20
dfcost=pd.DataFrame()
for nbclust in  range(1,nbclust_max):
    cluster_idx = scipy.cluster.hierarchy.fcluster(row_linkage,nbclust,criterion='maxclust')
    cluster = [y for y in cluster_idx]    
    new_df = pd.DataFrame(data = {"term": list_name, "cluster": cluster})
    new_df['nbseg']=nbclust
    new_df_count = new_df.groupby(['cluster']).agg(term_size=('term', 'size')).reset_index()
    new_df_count = new_df_count.loc[new_df_count['term_size'] == 1]
    if len(new_df_count)>0:
        break
        
    dfcost=dfcost.append(new_df)
    
#Liste des distances entre chaque paire de termes
df_sim['v1']=df_sim.index
df_sim_melt = df_sim.melt(id_vars=['v1'])
df_sim_melt.columns=['v1', 'v2', 'sim']
df_sim_melt['dist']=1-df_sim_melt['sim']
df_sim_melt

## Distance moyenne entre éléments de chaque cluster (avec nombre de cluster compris entre 1 et 20)

In [ ]:
df_dist_clust = pd.merge(df_sim_melt, dfcost, how='inner', left_on=['v1'], right_on=['term'])
df_dist_clust = pd.merge(df_dist_clust, dfcost, how='inner', left_on=['v2'], right_on=['term'])
df_dist_clust = df_dist_clust.loc[df_dist_clust['nbseg_x'] == df_dist_clust['nbseg_y']]
df_dist_clust = df_dist_clust.loc[df_dist_clust['cluster_x'] == df_dist_clust['cluster_y']]
#df_dist_clust['distsq']=df_dist_clust['dist']*df_dist_clust['dist']
df_dist_clust_inertia = df_dist_clust.groupby(['nbseg_x', 'cluster_x' ,'v1']).agg(dist_mean=('dist', 'mean')).reset_index()
df_dist_clust_inertia['distsq']=df_dist_clust_inertia['dist_mean'].pow(2)



## Somme des distances moyennes au carré intra-cluster

In [ ]:
df_dist_clust_inertia = df_dist_clust_inertia.groupby(['nbseg_x', 'cluster_x']).agg(dist_mean_sum=('distsq', 'sum')).reset_index()
df_dist_clust_inertia

## Somme des distances moyennes au carré inter-cluster

In [ ]:
df_dist_clust_inertia = df_dist_clust_inertia.groupby(['nbseg_x']).agg(dist_mean_sum=('dist_mean_sum', 'sum')).reset_index()
df_dist_clust_inertia

## Distribution de dist_mean_sum

In [ ]:
import plotly.express as px
fig = px.scatter(df_dist_clust_inertia, x='nbseg_x', y='dist_mean_sum', width=800, height=600)
fig

In [ ]:
from kneed import KneeLocator
x=df_dist_clust_inertia['nbseg_x'].values
y=df_dist_clust_inertia['dist_mean_sum'].values
kn = KneeLocator(x, y, curve='convex', direction='decreasing')
nb_segm_optimal=kn.knee
print(nb_segm_optimal)

dfcost = dfcost.loc[dfcost['nbseg'] == nb_segm_optimal]
dfcost = dfcost.sort_values(by=['cluster'], ascending=[True])

dfcost = dfcost.loc[dfcost['nbseg'] == nb_segm_optimal]
dfcost = dfcost.sort_values(by=['cluster'], ascending=[True])
dfcost

In [ ]:
import scipy.spatial as sp, scipy.cluster.hierarchy as hc

fig = plt.figure(1, figsize=(10, 20))
#th = 1.63
row_linkage = linkage(sp.distance.pdist(similarity), method='complete')# for x in (t.values, rho_terms.values.T))
dn = dendrogram(row_linkage, leaf_font_size= 12, labels=list_name, orientation = "left")
#plt.axvline(x=th, c='grey', lw=1, linestyle='dashed')

cluster_idx = scipy.cluster.hierarchy.fcluster(row_linkage,t= th,criterion="distance")
plt.savefig(f"clustermap/dendogram_{domain}_{segment}.png")

In [ ]:

cluster = [y for y in cluster_idx]
    
new_df = pd.DataFrame(data = {"term": list_name, "cluster": cluster})
new_df.sort_values("cluster", ascending = True)

dic_cluster = {1 : "Alternatives aux pesticides", 
               2 : "Contre le monde de Monsanto",  
               3 : "Médecins", 
               4 :"Résidus chimiques",
               5 : "Expertise et risque sanitaire",
               6 : "Les systèmes agricoles en question",
               7 : "Abeilles et néonicotinoïdes",
               8 : "OGM et herbicides",
               9 : "Générations futures",
               10 : "Les résidus de pesticides dans l'alimentation"
              }

print(new_df.sort_values("cluster", ascending = True))
#print(len(dic_cluster), len(cluster))


In [ ]:
nombre_texts = len(df3["text"])
start_segment = df1["start_segment"].iloc[1]#.strftime("%Y-%m-%d")
end_segment = df1["end_segment"].iloc[1]#.strftime("%Y-%m-%d")

for i, text in enumerate(df3["text"]):
    doc_id = df3["doc_id"].iloc[i]
    list_term = [x for x in df1["term"].loc[(df1["doc_id"] == doc_id)].unique()]
    term_to_search = ['victimes']
    nb_term_to_search = [x for x in term_to_search if x in list_term]
    
    if len(nb_term_to_search) >= 0:
        #Thematique = "Tous contre Monsanto"
        #df1.loc[(df1["doc_id"].str.contains(doc_id) == True), "Subject"] = Thematique
        list_top50_term = [x for x in df1["term"].loc[(df1["doc_id"] == doc_id) & (df1["in_top50_term"] == 1)].unique()]
        list_top50_term_absent = set(specific_terms) - set(list_top50_term)
        date = df3["date"].iloc[i]#.strftime("%Y-%m-%d")
        auteur = df3["source"].iloc[i]
        display(HTML(f"<h2 style='text-align:center;'> Texte {doc_id} ({i+1} sur {nombre_texts})</h2>"))
        display(HTML(f"<strong>Segment :</strong> {segment} ({start_segment} ; {end_segment})"))
        display(HTML(f"<strong>Date :</strong> {date}"))
        display(HTML(f"<strong>Auteur :</strong> {auteur} \n"))
        print(f"Termes indexés : {list_term} \n")
        print(f"Termes spécifiques présents : {list_top50_term} \n")
        print(f"Termes spécifiques absents : {list_top50_term_absent} \n")
        #display(HTML(f"<strong>Thématique :</strong> {Thematique} \n"))
        try:
            text1 = text
            for j, term in enumerate(list_term):
                if term not in list_top20_term:
                    text1 = re.sub(term, f'<mark style="background-color:yellow">{term}</mark>', text1, flags=re.I)  
                else:
                    text1 = re.sub(term, f'<mark style="background-color:pink">{term}</mark>', text1, flags=re.I)  
            display(HTML(f'{text1}'))
        except:
            print(f"{text} \n")
    else:
        pass
   

In [ ]:
new_df["cluster_name"] = new_df['cluster'].map(dic_cluster)  
new_df["origin"] = domain
new_df["segm"] = segment
new_df.sort_values("cluster", ascending = True)


In [ ]:
df_cluster = pd.read_csv(cluster_qualification, sep = "\t").drop(columns = ["Unnamed: 0"])

In [ ]:
df_cluster

In [ ]:
new_df2 = pd.concat([df_cluster, new_df])
new_df2

In [ ]:
new_df2.to_csv(f'{path_base}cluster_qualification.csv', sep = "\t") #cluster qualifié par origin et segment

## Facebook

### 1ère période : 16 novembre 2011 -- 31 janvier 2012:1ère période (Segment 0)

<img src="clustermap/cm_facebook_0.png" alt="segment0_FB" style="height: 100px; width:100px;"/>

##### 
La première période sur facebook semble être caractérisée par trois thématiques:

- La première concerne le procès opposant Paul François (céréalier charantais) à Monsanto. Paul François s'est retrouvé dans le coma après avoir nettoyé une cuve contenant du Lasso (produit commercialisé par Monsanto). Le débat a débuté le 12 décembre 2011.

> pesticides : un agriculteur en procès contre monsanto Ce lundi 12 décembre 2011, paul françois, agriculteur victime des pesticides, sera en procès contre monsanto, en première instance, au TGI de Lyon, dans l'espoir de faire reconnaître la faute de la firme et d'engager la responsabilité de cette dernière. bioaddict.fr Nous en appelons à toutes celles et ceux qui le souhaitent à se rendre au TGI de Lyon le 12 décembre à 13h30 pour témoigner de leur soutien à paul françois et, ainsi, à toutes les victimes des pesticides. [Auteur : BIO ADDICT]

- Un deuxième ensemble de textes évoque le problème du déclin des pollinisateurs et, plus spécifiquemet, des abeilles. Les "termes spécifiques" du segment 0 que l'on retrouve dans ces textes sont par exemple : *'effet', 'pesticides', 'miel', 'parasite'*. **"Parasites"** et **"pesticides"** font référence à deux des causes supposées de disparition des abeilles (les pesticides à travers les néonicotinoïdes et les parasites avec le Varoa). Une disparition qui se manifesterait notamment par une baisse de production du **miel**.


> Fin des abeilles = fin du monde ? Oui bon, c’est peut-être un peu exagéré. Quoique…sans les abeilles, l’équilibre écologique serait gravement menacé. En effet, les abeilles ont un rôle essentiel pour la survie des végétaux depuis 50 millions d’années [...] On a découvert que les abeilles sont toxico-sensibles et donc plus vulnérables que la plupart des autres insectes aux <mark>pesticides</mark>. [...] En France, le Gaucho de Bayer et le Régent de BASF, deux insecticides accusés de ravager les abeilles, ont été interdits en 2006 mais malgré tout l’hécatombe se poursuit dans les ruches. On ne sait pas bien pourquoi et cela est très préoccupant. <mark>Certains apiculteurs soupçonnent que les pesticides</mark> toujours utilisés par l’agriculture sont à l’origine de ce qu’on appelle désormais le « syndrome d’effondrement des colonies », maladie dont on ne sait pas encore grand-chose. [...] Récemment, <mark>d’autres explications</mark> ont été données quant à la disparition des abeilles : 
> - <mark>Le frelon asiatique</mark> est montré du doigt. Cet insecte venu de l’étranger est un prédateur féroce pour les abeilles puisqu’il se nourrit de leurs larves. 
> - <mark>Une mouche parasite</mark> est également mise en cause depuis peu. Une étude américaine publiée en janvier 2012 a en effet mis en lumière le rôle néfaste de la mouche Apocephalus borealis. [Auteur: Let's save our Earth - Sauvons notre planète]

- Un troisième ensemble de textes porte sur le rapport publié par Génération futures sur les  résidus de pesticides dans l'alimentation

> Choisir de manger bio, c'est protéger sa santé L'association générations futures a effectué des tests comparant un menu conventionnel et un menu bio : le résultat est sans appel, le menu bio ne contenant quasiment aucun résidus de pesticides. "Il y a 223 fois moins de résidus de pesticides en moyenne dans les aliments bio analysés que dans les a... bioaddict.fr Mangez bio, c'est bon pour la santé ! Et ça, c'est vachement bien ! [Auteur : Les 2 vaches, industrie agroalimentaire]

- On trouve ensuite les textes qui prennent pour cible "Monsanto et son monde" ainsi que les effets sanitaires des produits commercialisés par la firme. Il s'agit probablement d'un ensemble de réactions à la suite de la sortie du film éponyme *Le monde selon Monsanto*.

> *Le monde selon Monsanto* : Ce film retrace l'histoire de monsanto, une multinationale américaine, aujourd'hui leader mondial des ogm, et considérée comme l'un des plus grands pollueurs de l'ère industrielle (PCB, agent orange, hormones de croissance, roundup...). Après une enquête de trois ans, en Amérique du nord et du sud, en europe et en Asie, il reconstitue la genèse d'un empire industriel, qui, à grand renfort de rapports mensongers, de collusion avec l'administration nord-américaine, de pressions et tentatives de corruption, est devenu l'un des premiers semenciers de la planète.

Il est intéressant de noter que Monsanto (et d'autres firmes comme Syngenta ou Bayer) semble être le lien entre différents "cluster": l'affaire Paul François, les débats liés au Roundup.

- Enfin, de nombreux posts évoquent les "pesticides" de façon générale sans faire de lien à une affaire en particulier.

## Test (annexe)

In [ ]:
text = df2['text'].loc[(df2["doc_id"] == "413916")]
for x in text :
    print(x)

In [ ]:
text = """Santé Canada reçoit l'ordre de réexaminer les effets environnementaux du Roundup | equiterre.org 
- P La Cour fédérale du Canada a ordonné à Santé Canada de réévaluer sa décision de ne pas examiner les effets 
d'un herbicide, commercialisé sous le nom de Roundup. Ce pesticide est un des plus populaires sur le marché et 
il accompagne une proportion importante des plantes modifiées génétiquement utilis... equiterre.org 
Santé Canada reçoit l'ordre de la Cour fédérale de réexaminer les effets environnementaux du RoundUp!"""

In [ ]:
list_term = ["pesticides", "roundup"]

In [ ]:
import re
for j, term in enumerate(list_term):
    if term[-1]== "s":
        singular_term = term[0:-1]
    else:
        singular_term = term
    #text = text.replace(term[0:len(term)-1], f'<mark style="background-color:yellow">{term}</mark>')
    text = re.sub(term, f'<mark style="background-color:yellow">{term}</mark>', text, flags=re.I)          #

display(HTML(f'{text}'))